In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 0. 패키지

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
import re

load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDIGN_MODEL = 'text-embedding-3-large' 
UPSTAGE_EMBEDDIGN_MODEL = 'solar-embedding-1-large-passage'

PINECONE_INDEX_NAME = 'law'

COHERE_API_KEY = os.getenv('COHERE_API_KEY')

# 1. 데이터 생성

## law

In [96]:
loader = Docx2txtLoader("data/raw/law/주택임대차보호법_법률_21065_20260102.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

law_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/law"})
    for part in parts
    if part.strip()
]

len(law_list)

43

In [65]:
law_list[42].page_content

'제31조(벌칙 적용에서 공무원 의제) 공무원이 아닌 주택임대차위원회의 위원 및 주택임대차분쟁조정위원회의 위원은 「형법」 제127조, 제129조부터 제132조까지의 규정을 적용할 때에는 공무원으로 본다.\n\n[본조신설 2016. 5. 29.]\n\n\n\n\n\n\n\n부칙 <제21065호,2025. 10. 1.>(정부조직법)\n\n제1조(시행일) 이 법은 공포한 날부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률 중 이 법 시행 전에 공포되었으나 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터 시행하고, 다음 각 호의 개정규정은 해당 호에서 정하는 날부터 시행한다.\n\n1. 다음 각 목의 개정규정은 2026년 1월 2일부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률(가목 및 나목의 개정규정과 관련되는 부분으로 한정한다) 중 본문에 따른 시행일 전에 공포되었으나 본문에 따른 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터 시행한다.\n\n가. 제19조제4항, 제23조, 제29조제1항제1호 및 제30조의 개정규정\n\n나. 제12조제2항, 제19조제3항, 제22조 및 제29조제2항 단서의 개정규정(재정경제부장관 및 재정경제부에 관한 부분으로 한정한다)\n\n다. 부칙 제7조에 따라 개정되는 법률(가목 및 나목의 개정규정과 관련되는 부분으로 한정한다)\n\n2. 생략\n\n제2조 부터 제6조까지 생략\n\n제7조(다른 법률의 개정) ①부터 <120>까지 생략\n\n<121> 주택임대차보호법 일부를 다음과 같이 개정한다.\n\n제8조의2제4항제3호 중 “기획재정부”를 “재정경제부”로 한다.\n\n<122>부터 <626>까지 생략\n\n제8조 생략\n\n\n\n법제처\n\n\n\n \n\n국가법령정보센터'

In [79]:
loader = Docx2txtLoader("data/raw/law/주택임대차보호법_시행령_35947_20260102.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

enforcement_decree_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/law"})
    for part in parts
    if part.strip()
]

len(enforcement_decree_list)

37

In [61]:
enforcement_decree_list[36].page_content

'제35조(조정결과의 통지) ① 조정위원회는 조정절차가 종료되면 그 결과를 당사자에게 통지하여야 한다.\n\n② 조정위원회는 법 제26조제4항에 따른 조정서가 작성된 경우 조정서 정본을 지체 없이 당사자에게 교부 또는 송달하여야 한다.\n\n[본조신설 2017. 5. 29.]\n\n\n\n\n\n\n\n부칙 <제35947호,2025. 12. 30.>(재정경제부 직제)\n\n제1조(시행일) 이 영은 2026년 1월 2일부터 시행한다. 다만, 부칙 제6조에 따라 개정되는 대통령령 중 이 영 시행 전에 공포되었으나 시행일이 도래하지 않은 대통령령을 개정한 부분은 각각 해당 대통령령의 시행일부터 시행한다.\n\n제2조 부터 제5조까지 생략\n\n제6조(다른 법령의 개정) ①부터 <108>까지 생략\n\n<109> 주택임대차보호법 시행령 일부를 다음과 같이 개정한다.\n\n제17조제3항제1호 중 “기획재정부”를 “재정경제부”로 한다.\n\n<110>부터 <313>까지 생략\n\n\n\n법제처\n\n\n\n \n\n국가법령정보센터'

In [80]:
loader = Docx2txtLoader("data/raw/law/민법_임대차_발췌.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

civil_law_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/law"})
    for part in parts
    if part.strip()
]

len(civil_law_list)

38

In [87]:
civil_law_list[33].page_content

'제650조(임차건물등의 부속물에 대한 법정질권) 건물 기타 공작물의 임대인이 임대차에 관한 채권에 의하여 그 건물 기타 공작물에 부속한 임차인소유의 동산을 압류한 때에는 질권과 동일한 효력이 있다.'

## rule

In [97]:
loader = Docx2txtLoader("data/raw/rule/주택임대차보호법_시행규칙.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

enforcement_rules_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/rule"})
    for part in parts
    if part.strip()
]

len(enforcement_rules_list)

13

In [98]:
enforcement_rules_list[12].page_content

'제8조(수수료) ① 영 제7조제1항에 따른 수수료의 금액은 다음과 같다. 다만, 수수료를 계산할 때 100원 미만 금액은 계산하지 아니하고, 열람과 동시에 출력서면을 제공하는 경우에는 열람에 관한 정보제공 수수료를 별도로 계산하지 아니한다. <개정 2013. 12. 31.>\n\n1. 확정일자 부여 수수료: 1건마다 600원(계약증서가 4장을 초과할 경우 초과 4장마다 100원 추가)\n\n2. 정보제공 수수료: 1건마다 600원(출력물이 10장을 초과할 경우 초과 1장마다 50원 추가)\n\n② 제1항에도 불구하고 공증인에 대한 수수료는 「공증인 수수료 규칙」에 따른다.<신설 2013. 12. 31.>\n\n③ 영 제7조제2항에서 “「국민기초생활 보장법」에 따른 수급자 등 법무부령으로 정하는 사람”이란 다음 각 호의 어느 하나에 해당하는 사람을 말한다.<개정 2013. 12. 31., 2016. 5. 23., 2018. 10. 26.>\n\n1. 「국민기초생활 보장법」 제2조제2호에 따른 수급자\n\n2. 「독립유공자예우에 관한 법률」 제6조에 따라 등록된 독립유공자 또는 그 유족(선순위자만 해당된다)\n\n3. 「국가유공자 등 예우 및 지원에 관한 법률」 제6조에 따라 등록된 국가유공자 또는 그 유족(선순위자만 해당된다)\n\n4. 「고엽제후유의증 환자지원 등에 관한 법률」 제4조에 따라 등록된 고엽제후유증환자, 고엽제후유의증환자 또는 고엽제후유증 2세환자\n\n5. 「참전유공자예우 및 단체설립에 관한 법률」 제5조에 따라 등록된 참전유공자\n\n6. 「5ㆍ18민주유공자예우에 관한 법률」 제7조에 따라 등록 결정된 5ㆍ18민주유공자 또는 그 유족(선순위자만 해당된다)\n\n7. 「특수임무수행자 지원 및 단체설립에 관한 법률」 제6조에 따라 등록된 특수임무수행자 또는 그 유족(선순위자만 해당된다)\n\n8. 「의사상자 등 예우 및 지원에 관한 법률」 제5조에 따라 인정된 의상자 또는 의사자유족(선순위자만 해당된다)\n\n9. 「한부모가족지원법」 제5조에 따

In [110]:
loader = Docx2txtLoader("data/raw/rule/확정일자_대법원규칙_2986_20210610.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

confirmation_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/rule"})
    for part in parts
    if part.strip()
]

len(confirmation_list)

15

In [111]:
confirmation_list[14].page_content

'제14조(대법원예규 위임) 확정일자 부여 등의 절차와 관련하여 필요한 사항 중 이 규칙에서 정하고 있지 아니한 사항은 대법원예규로 정할 수 있다.\n\n\n\n\n\n\n\n부칙 <제2986호,2021. 5. 27.>(부동산등기규칙)\n\n제1조(시행일) 이 규칙은 2021년 6월 10일부터 시행한다.\n\n제2조 생략\n\n제3조(다른 규칙의 개정) ①부터 ③까지 생략\n\n④ 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙 일부를 다음과 같이 개정한다.\n\n제6조제4항제1호 중 “것을”을 “것으로서 법원행정처장이 지정ㆍ공고하는 인증서를”로 하고, 같은 조에 제5항을 신설한다.\n\n⑤ 제4항제1호의 공고는 인터넷등기소에 하여야 한다.\n\n\n\n법제처\n\n\n\n \n\n국가법령정보센터'

## case

In [20]:
loader = Docx2txtLoader("data/raw/case/전세피해법률상담사례집.docx")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,   
                                               chunk_overlap=200) 
legal_counsel_list = loader.load_and_split(text_splitter=text_splitter)
len(legal_counsel_list)

41

In [22]:
legal_counsel_list[40].page_content

'3. 대응방안\n\n임차인이 임차주택을 경매할 때에는 주택을 인도하지 않아도 되며, 임대인의 다른 재산을 강제집행 할 때에는 주택을 인도해야 합니다. 또한 최선순위 임차인이라서 보증금을 전액배당 받을 때에는 최고가매수신고인이 작성한 "인도확인서"의 제출이 필요하지만, 일부만을 배당받을 때에는 "인도확인서"는 필요하지 않습니다.\n\n만약 임차인이 최선순위 임차인인데 경락대금으로부터 못 받은 미변제금이 있다면 대항력을 행사하여 경락인으로부터 전액을 다 받을 때까지 인도를 거절할 수가 있으나, 최선순위 대항력이 없는 임차인이라면 배당기일 이후에는 집을 인도해야 합니다'

In [23]:
loader = Docx2txtLoader("data/raw/case/전세피해지원사례집_2025.docx")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,   
                                               chunk_overlap=200) 
legal_support_list = loader.load_and_split(text_splitter=text_splitter)
len(legal_support_list)

7

In [24]:
legal_support_list[6].page_content

'회생·파산절차 관련 법률상담이 필요한 경우 HUG 전세피해지원센터 또는 지자체 전세피해지원센터를 통해 무료 상담을 받을 수 있습니다.\n\n\n\n\n\n21. 형사고소에 따른 무고죄\n\n임대인이 보증금을 돌려주지 않아 형사고소 하고자 하니, 제가 고소하면 임대인이 저를 무고죄로 맞고소하겠다고 합니다.\n\n전세사기로 의심되는 피해를 당한 경우, 피해사실이 인정된다면 형사고소를 할 수 있으며, 수사 결과 사기죄가 불성립된다고 하여 무조건 무고죄로 처벌되는 것은 아닙니다.\n다만, 형사처벌 등을 받게 할 목적으로 허위사실을 신고한다면 무고죄가 성립될 가능성은 있습니다.\n\n이에 대해 법률상담이 필요한 경우 HUG 전세피해지원센터(서울·강서) 또는 **지자체 전세피해지원센터(인천·경기·대전·대구·부산·광주)**를 통해 무료 법률상담을 받을 수 있습니다.\n\n국토부 전세사기피해자(나목·다목 포함) 또는 HUG 전세피해확인서를 발급받은 임차인은 소송대리 법률구조를 통해 형사사건을 포함하여 보증금 피해 회복을 위한 포괄적인 법률지원을 받을 수 있습니다.\n\n※ 중위소득 125% 초과 피해자는 대한법률구조공단을 통한 변호사 수임료를 최대 250만 원까지 지원받을 수 있으며, 중위소득 125% 이하 피해자는 법률구조공단 무료소송 지원 대상이 됩니다.\n\n\n\n\n\n22. 다가구주택 보증금 허위 고지 문제\n\n공인중개사가 선순위 보증금을 속이고 계약을 유도해 전세피해를 받았습니다. 공인중개사의 책임은 없나요?\n\n공인중개사는 중개물건과 관련하여 설명의무가 없는 사항이라 하더라도, 임차인이 계약 여부를 결정하는 데 중요한 사안이라면 거짓된 정보를 진실인 것처럼 전달해서는 안 됩니다.\n(대법원 2022다212694, 공제청구)\n\n공인중개사의 기망행위로 인해 보증금 손실이 발생하였고 이에 대한 책임 입증이 가능한 경우,\n민사소송 등을 통해 공인중개사의 일반 재산에 대한 강제집행이 가능하며,\n한국공인중개사협회에 공제금 청구를 통해 피해회복이 일부 가능합니

In [83]:
embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)

# 2. 함수

In [13]:
import re

def remove_special_chars(text:str) -> str:
    '특수문자 및 \n제거(:는 그대로)'
    # \n 제거
    text = text.replace('\n',' ')
    
    # 한글, 영문, 숫자, 공백, 마침표, 콤마, :만 남기고 전부 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s,.:]', '', text) 
    
    # 불필요한 중복 공백을 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

In [14]:
# 제목을 추출하는 함수(exaone3.5) : cmd창에서 ollama pull exaone3.5:2.4b

from langchain_ollama import ChatOllama

def extract_title_with_llm(content):
    'exaone3.5를 사용하여 content의 제목을 추출하는 함수'
    title_extractor_llm = ChatOllama(model='exaone3.5:2.4b',
                                     temperature=0.1,
                                     num_predict=30, # 최대 30토큰까지만 출력
                                     )
    # content = remove_special_chars(content)
    prompt = f'''다음 소득세법 조문의 핵심 제목을 30토큰 이내로 간단히 완벽하게 말이 되도록 추출하세요. 중간에 말이 끊기면 안됩니다
    조문 : {content}'''
    ai_message = title_extractor_llm.invoke(prompt)
    title = ai_message.content.strip()
    return remove_special_chars(title)

In [ ]:
"""
전월세 계약서 독소조항 탐지를 위한 Pinecone 카테고리 분류기
주택임대차보호법 기반
"""

def get_categories():
    """
    카테고리별 키워드 매핑
    Pinecone 메타데이터의 'category' 필드에 사용
    """
    return {
        '대항력': [
            '대항력', '대항요건', '주택인도', '인도', '주민등록', '전입신고', 
            '점유', '제삼자', '효력발생', '입주', '거주', '전입', '주민센터'
        ],
        
        '우선변제권': [
            '우선변제권', '확정일자', '보증금반환청구', '후순위권리자', '환가대금',
            '변제', '경매', '공매', '우선변제', '배당', '보증금회수', '임대차계약증서'
        ],
        
        '임차권등기': [
            '임차권등기명령', '임차권등기', '임차권', '등기명령', '대항력유지',
            '임대차등기', '등기촉탁', '지방법원', '소명', '등기신청'
        ],
        
        '계약갱신': [
            '계약갱신청구권', '묵시적갱신', '갱신거절', '계약갱신', '갱신',
            '갱신요구', '계약해지', '해지통지', '갱신거절통지', '계약조건변경',
            '임대차기간', '2년', '계약만료', '계약종료', '실거주', '직계존비속'
        ],
        
        '차임보증금': [
            '보증금', '차임', '보증금반환', '차임액', '증액', '증액청구',
            '증액비율', '월차임', '월차임전환', '산정률', '5퍼센트', '초과차임',
            '전세금', '임대료', '보증금증액', '연체', '2기차임', '차임연체'
        ],
        
        '임대인의무': [
            '정보제시의무', '납세증명서', '미납국세', '미납지방세', '체납',
            '확정일자정보', '선순위임차인', '임대인동의', '정보제공요청'
        ],
        
        '소액임차인': [
            '소액보증금', '최우선변제', '소액임차인', '우선변제금액',
            '담보물권', '담보물권자', '선순위', '최선순위', '배당순위'
        ],
        
        '보증금보험': [
            '전세보증금반환보증보험', '보증보험', '주택도시보증공사',
            '한국주택금융공사', '보증가입', '보증금보험', '보증금반환의무', '전세사기'
        ],
        
        '전대양도': [
            '전대', '전대금지', '전대차', '임대인동의', '무단전대',
            '양도', '전매', '권리양도', '명의변경'
        ],
        
        '건축물': [
            '건축물대장', '불법건축물', '위반건축물', '보존등기',
            '미등기부동산', '이행강제금', '건축허가', '용도', '면적',
            '주택현황', '등기부등본', '등기부'
        ],
        
        '담보권리': [
            '근저당권', '저당권', '가등기', '선순위권리', '채권최고액',
            '담보가등기', '후순위', '권리순서', '말소기준권리',
            '소유권이전등기', '권리관계', '등기사항'
        ],
        
        '강제집행': [
            '가압류', '가처분', '경매신청', '공매처분', '낙찰', '경락',
            '강제집행', '인도명령', '명도소송', '압류', '집행개시',
            '체납처분', '무잉여기각', '매각'
        ],
        
        '신탁대리': [
            '신탁등기', '수탁자', '위탁자', '대리권', '자기계약',
            '쌍방대리', '무권대리', '표현대리', '신탁재산', '신탁원부'
        ],
        
        '중개업무': [
            '중개대상물확인설명서', '공인중개사', '중개보조원', '불법중개',
            '중개수수료', '중개보수', '중개의뢰인', '중개업무', '공제보증보험'
        ],
        
        '분쟁조정': [
            '주택임대차분쟁조정위원회', '조정위원회', '분쟁조정', '조정신청',
            '조정절차', '조정안', '조정서', '조정성립', '민사조정법'
        ],
        
        '주택유형': [
            '다가구주택', '다세대주택', '단독주택', '공동주택', '집합건물',
            '주거용건물', '주택', '임차주택', '주택일부', '연립', '빌라', '아파트'
        ],
        
        '독소조항': [
            '임차인책임', '임대인면책', '손해배상', '위약', '특약', '포기',
            '면제', '제한', '금지', '거부', '불가', '일방적', '변경불가',
            '예외', '단서', '제외', '원상복구', '수리비', '관리비', '공과금',
            '전액부담', '동의없이', '승낙없이', '사전동의', '즉시', '무효'
        ]
    }


def classify_text(text):
    """
    텍스트를 분석하여 해당하는 카테고리들을 반환
    
    Args:
        text (str): 분류할 텍스트 (계약 조항, 법률 용어 등)
    
    Returns:
        list: 매칭된 카테고리 리스트 (점수 높은 순)
    
    Example:
        >>> classify_text("임차인은 주택을 인도받고 전입신고를 완료하였다")
        ['대항력', '주택유형']
    """
    categories = get_categories()
    category_scores = {}
    
    text_lower = text.lower()
    
    for category, keywords in categories.items():
        score = 0
        matched_keywords = []
        
        for keyword in keywords:
            if keyword.lower() in text_lower:
                score += 1
                matched_keywords.append(keyword)
        
        if score > 0:
            category_scores[category] = {
                'score': score,
                'matched_keywords': matched_keywords
            }
    
    # 점수순 정렬
    sorted_categories = sorted(
        category_scores.items(), 
        key=lambda x: x[1]['score'], 
        reverse=True
    )
    
    return [cat for cat, info in sorted_categories]


def get_primary_category(text):
    """
    텍스트의 주요 카테고리 1개만 반환 (점수가 가장 높은 것)
    
    Args:
        text (str): 분류할 텍스트
    
    Returns:
        str: 주요 카테고리 (매칭 없으면 '기타')
    """
    categories = classify_text(text)
    return categories[0] if categories else '기타'


def create_pinecone_metadata(text, doc_id, law_article=None, additional_info=None):
    """
    Pinecone에 넣을 메타데이터 생성
    
    Args:
        text (str): 원문 텍스트
        doc_id (str): 문서 ID
        law_article (str, optional): 관련 법조문
        additional_info (dict, optional): 추가 정보
    
    Returns:
        dict: Pinecone 메타데이터
    
    Example:
        >>> create_pinecone_metadata(
        ...     text="임차인은 계약갱신을 청구할 수 있다",
        ...     doc_id="law_article_6-3",
        ...     law_article="제6조의3"
        ... )
        {
            'text': '임차인은 계약갱신을 청구할 수 있다',
            'category': '계약갱신',
            'all_categories': ['계약갱신'],
            'law_article': '제6조의3',
            'doc_id': 'law_article_6-3'
        }
    """
    metadata = {
        'text': text,
        'category': get_primary_category(text),  # 주 카테고리
        'all_categories': classify_text(text),   # 모든 관련 카테고리
        'doc_id': doc_id
    }
    
    if law_article:
        metadata['law_article'] = law_article
    
    if additional_info:
        metadata.update(additional_info)
    
    return metadata


def batch_categorize(texts):
    """
    여러 텍스트를 한번에 분류
    
    Args:
        texts (list): 텍스트 리스트
    
    Returns:
        list: 각 텍스트의 카테고리 정보
    """
    results = []
    for i, text in enumerate(texts):
        results.append({
            'index': i,
            'text': text,
            'primary_category': get_primary_category(text),
            'all_categories': classify_text(text)
        })
    return results


# ============= Pinecone 사용 예시 =============

def example_pinecone_usage():
    """
    실제 Pinecone 사용 예시 코드
    """
    print("=" * 60)
    print("Pinecone 메타데이터 생성 예시")
    print("=" * 60)
    
    # 예시 1: 법률 조항
    sample_texts = [
        {
            'text': '임차인이 주택의 인도와 주민등록을 마친 때에는 그 다음 날부터 제삼자에 대하여 효력이 생긴다',
            'doc_id': 'law_article_3',
            'law_article': '제3조 제1항'
        },
        {
            'text': '확정일자를 갖춘 임차인은 후순위권리자나 그 밖의 채권자보다 우선하여 보증금을 변제받을 권리가 있다',
            'doc_id': 'law_article_3-2',
            'law_article': '제3조의2 제2항'
        },
        {
            'text': '임차인은 계약갱신을 요구할 경우 정당한 사유 없이 거절하지 못한다',
            'doc_id': 'law_article_6-3',
            'law_article': '제6조의3 제1항'
        },
        {
            'text': '차임 또는 보증금의 증액은 청구 당시 차임 또는 보증금의 100분의 5를 초과하지 못한다',
            'doc_id': 'law_article_7',
            'law_article': '제7조 제1항'
        }
    ]
    
    print("\n[법률 조항 분류 결과]\n")
    for item in sample_texts:
        metadata = create_pinecone_metadata(
            text=item['text'],
            doc_id=item['doc_id'],
            law_article=item['law_article']
        )
        print(f"📄 {item['law_article']}")
        print(f"   카테고리: {metadata['category']}")
        print(f"   관련: {', '.join(metadata['all_categories'][:3])}")
        print()
    
    # 예시 2: 계약서 독소조항
    print("\n[독소조항 분류 결과]\n")
    
    toxic_clauses = [
        '임차인은 전입신고를 하지 않기로 한다',
        '보증금은 임대인의 판단에 따라 반환하지 않을 수 있다',
        '계약갱신청구권을 행사할 수 없다',
        '임대인은 언제든지 계약을 해지할 수 있다',
        '차임은 매년 10% 인상한다'
    ]
    
    for i, clause in enumerate(toxic_clauses):
        metadata = create_pinecone_metadata(
            text=clause,
            doc_id=f'toxic_clause_{i+1}',
            additional_info={'is_toxic': True}
        )
        print(f"⚠️  독소조항 {i+1}")
        print(f"   내용: {clause}")
        print(f"   카테고리: {metadata['category']}")
        print(f"   관련: {', '.join(metadata['all_categories'])}")
        print()
    
    # 예시 3: Pinecone 업서트 형식
    print("\n[Pinecone 업서트 포맷]\n")
    
    # 실제 사용할 때는 이렇게
    pinecone_vectors = []
    
    for item in sample_texts[:2]:  # 예시로 2개만
        metadata = create_pinecone_metadata(
            text=item['text'],
            doc_id=item['doc_id'],
            law_article=item['law_article']
        )
        
        # 벡터는 임베딩 모델로 생성 (예시로 더미 벡터)
        dummy_vector = [0.1] * 1536  # OpenAI embedding 차원
        
        pinecone_vectors.append({
            'id': metadata['doc_id'],
            'values': dummy_vector,
            'metadata': metadata
        })
    
    print("```python")
    print("# Pinecone에 업서트하는 실제 코드")
    print("index.upsert(vectors=[")
    for vec in pinecone_vectors:
        print(f"    {{")
        print(f"        'id': '{vec['id']}',")
        print(f"        'values': [...],  # 임베딩 벡터 1536차원")
        print(f"        'metadata': {vec['metadata']}")
        print(f"    }},")
    print("])")
    print("```")
    
    # 예시 4: 카테고리별 필터링 쿼리
    print("\n[Pinecone 필터 쿼리 예시]\n")
    print("```python")
    print("# 대항력 관련 문서만 검색")
    print("results = index.query(")
    print("    vector=query_embedding,")
    print("    filter={'category': '대항력'},")
    print("    top_k=5")
    print(")")
    print()
    print("# 독소조항만 검색")
    print("results = index.query(")
    print("    vector=query_embedding,")
    print("    filter={'category': '독소조항'},")
    print("    top_k=10")
    print(")")
    print("```")


if __name__ == "__main__":
    # 사용 예시 실행
    example_pinecone_usage()
    
    print("\n" + "=" * 60)
    print("빠른 테스트")
    print("=" * 60)
    
    test_text = "임차인이 주택을 인도받고 주민등록을 마친 경우 대항력이 발생한다"
    
    print(f"\n테스트 텍스트: {test_text}")
    print(f"주요 카테고리: {get_primary_category(test_text)}")
    print(f"모든 카테고리: {classify_text(test_text)}")
    
    metadata = create_pinecone_metadata(
        text=test_text,
        doc_id='test_1',
        law_article='제3조'
    )
    
    print(f"\n생성된 메타데이터:")
    import json
    print(json.dumps(metadata, ensure_ascii=False, indent=2))


In [15]:
def categorize_content(content, top_k=None):
    '''
    내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters : 
        - content : 분류할 텍스트 내용
        - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
        
    Returns :
        - 카테고리 리스트(점수 높은 순)
    '''
    category_keywords = get_category()
    category_scores = {}
    
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0 :
            category_scores[category] = score
    
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x : x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    
    # 매칭되는 카테고리가 없으면 '기타' 반환
    if not all_categories:
        all_categories = ['기타']
        
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    return all_categories

In [16]:
# 해당 조항 추출하기

def get_article(content):
    '조항들 추출'
    article_match = re.findall(r'제(\d+)조', content)
    article_match =list(set(article_match)) # 중복 제거
    article_match.sort(key=lambda x : int(x))
    if article_match:
        return '「'+ ', '.join([f'{a}조' for a in article_match]) +'」'

# 3. metadata 추가하기

## law

In [105]:
# 주택임대차보호법_법률
law_enhanced_chunks = []
for i, chunk in enumerate(law_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    # metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 1
    metadata['src_title'] = '주택임대차보호법_법률(제21065호)(20260102)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    law_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(law_enhanced_chunks))

확장된 chunk 처리 완료
43


In [106]:
# 주택임대차보호법_시행령
enforcement_decree_enhanced_chunks = []
for i, chunk in enumerate(enforcement_decree_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    # metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 2
    metadata['src_title'] = '주택임대차보호법_시행령(대통령령)(제35947호)(20260102)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    enforcement_decree_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(enforcement_decree_enhanced_chunks))

확장된 chunk 처리 완료
37


In [107]:
# 민법 임대차
civil_enhanced_chunks = []
for i, chunk in enumerate(civil_law_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    # metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 4
    metadata['src_title'] = '민법_임대차(제20432호)(20260101)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    civil_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(civil_enhanced_chunks))

확장된 chunk 처리 완료
38


In [108]:
%%time

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

all_docs = (
    law_enhanced_chunks
    + enforcement_decree_enhanced_chunks
    + civil_enhanced_chunks
)

# 데이터를 처음 업로드할 때
index_name = 'law'
database = PineconeVectorStore.from_documents(documents=all_docs,
                                              embedding=embedding,
                                              index_name=index_name)

# 업로드한 벡터DB를 가져올 때
# database = PineconeVectorStore(embedding=embedding, # 질문을 임베딩하여 유사도 검색
#                                index_name=index_name)

CPU times: total: 3.7 s
Wall time: 22.8 s


## rule

In [109]:
# 주택임대차보호법_시행규칙
enforcement_rules_enhanced_chunks = []
for i, chunk in enumerate(enforcement_rules_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    # metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 3
    metadata['src_title'] = '주택임대차보호법_시행규칙(제1022호)(20220207)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    enforcement_rules_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(enforcement_rules_enhanced_chunks))

확장된 chunk 처리 완료
13


In [112]:
# 확정일자 대법원 규칙
confirmation_enhanced_chunks = []
for i, chunk in enumerate(confirmation_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    # metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 3
    metadata['src_title'] = '확정일자_대법원규칙(제2986호)(20210610)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    confirmation_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(confirmation_enhanced_chunks))

확장된 chunk 처리 완료
15


In [113]:
%%time

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

all_docs = (
    enforcement_rules_enhanced_chunks
    + confirmation_enhanced_chunks
)

# 데이터를 처음 업로드할 때
index_name = 'rule'
database = PineconeVectorStore.from_documents(documents=all_docs,
                                              embedding=embedding,
                                              index_name=index_name)

# 업로드한 벡터DB를 가져올 때
# database = PineconeVectorStore(embedding=embedding, # 질문을 임베딩하여 유사도 검색
#                                index_name=index_name)

CPU times: total: 1 s
Wall time: 8.67 s


## case

In [ ]:
# 확정일자 대법원 규칙
confirmation_enhanced_chunks = []
for i, chunk in enumerate(legal_counsel_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    # metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 7
    metadata['src_title'] = ''
    article_match = get_article(content)
    if article_match:
        metadata['related_article'] = article_match
    legal_counsel_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) 
print('확장된 chunk 처리 완료')
print(len(legal_counsel_enhanced_chunks))

In [ ]:
%%time

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

all_docs = (
    enforcement_rules_enhanced_chunks
    + confirmation_enhanced_chunks
)

# 데이터를 처음 업로드할 때
index_name = 'rule'
database = PineconeVectorStore.from_documents(documents=all_docs,
                                              embedding=embedding,
                                              index_name=index_name)

# 업로드한 벡터DB를 가져올 때
# database = PineconeVectorStore(embedding=embedding, # 질문을 임베딩하여 유사도 검색
#                                index_name=index_name)